## Create a Knowledge Base with Hierarchical Chunking Strategy
#### What will we do in this workshop?
1. Create a Knowledgebase (KB) in the vector database.
2. We will create a data source for the KB. The data source will be the Amazon Science and 10K documents stored in S3.
3. We will ingest the data from S3, use Hierarchical Chunking to chunk the data, generate vector embeddings, and store the chunks and their corresponding vector embeddings in the KB.
4. We will then ask some questions and query the KB to return some chunks and inspect relevancy score.
<br>Note: We are not sending the query and its chunks to a LLM in this notebook. We will do that in other notebooks.

#### Concept

**Hierarchical Chunking**: Organizes your data into a hierarchical structure, allowing for more granular and efficient retrieval based on the inherent relationships within your data. 

Organizing your data into a hierarchical structure enables your **RAG (Retrieval-Augmented Generation)** workflow to efficiently navigate and retrieve information from complex, nested datasets. After documents are parsed, the first step is to **chunk** them based on the **parent** and **child chunking size**. 

- **Parent chunks (higher level)** represent larger segments, such as entire documents or sections.
- **Child chunks (lower level)** represent smaller segments, such as paragraphs or sentences.

The relationship between parent and child chunks is maintained, allowing for **efficient retrieval and navigation** of the corpus.

#### Benefits

- **Efficient Retrieval**: The hierarchical structure enables faster and more targeted retrieval of relevant information by first performing a **semantic search** on child chunks and then returning the parent chunk. By replacing child chunks with parent chunks, we provide **larger and more comprehensive context** to the foundation model (FM).
- **Context Preservation**: Organizing the corpus hierarchically helps maintain contextual relationships between chunks, ensuring more **coherent and contextually relevant** text generation.

> **Note:** In hierarchical chunking, **parent chunks** are returned while **search is performed on child chunks**. As a result, you may see **fewer search results**, since one parent can have multiple child chunks.

### **Best Use Cases**
Hierarchical chunking is best suited for **complex documents** with a nested or hierarchical structure, such as:
- **Technical manuals**
- **Legal documents**
- **Academic papers** with complex formatting and nested tables.


In [79]:
# Import a module with few helper functions. 
# These functions will help us create knowledge base (KB), create data source for KB, and ingest data using semantic chunking to KB.

import importlib
import advanced_rag_utils

# Reload module
importlib.reload(advanced_rag_utils)

# Re-import all functions
from advanced_rag_utils import *

from datetime import datetime, timedelta, UTC

notebook_start_time = datetime.now(UTC)

In [80]:
# Let's load the variables we saved in the first notebook. We will use these variables
import json
with open("../variables.json", "r") as f:
    variables = json.load(f)

variables

{'accountNumber': '270597685972',
 'regionName': 'us-west-2',
 'collectionArn': 'arn:aws:aoss:us-west-2:270597685972:collection/3ethft3xms9as2092ulg',
 'collectionId': '3ethft3xms9as2092ulg',
 'vectorIndexName': 'ws-index-',
 'bedrockExecutionRoleArn': 'arn:aws:iam::270597685972:role/advanced-rag-workshop-bedrock_execution_role-us-west-2',
 's3Bucket': '270597685972-us-west-2-advanced-rag-workshop',
 'kbFixedChunk': 'SN9KSOQPOV',
 'kbSemanticChunk': 'KMZYCTNSWW',
 'kbHierarchicalChunk': 'V8EJKFPYTK',
 'kbCustomChunk': 'G8P2D7M28S',
 'sagemakerLLMEndpoint': 'endpoint-llama-3-2-3b-instruct-2025-05-02-18-22-06'}

In [81]:
# Load the dataframe related to costs from a csv file (if it already exists)
df_costs = load_df_from_csv()
df_costs

Loaded existing file: /home/sagemaker-user/sample-advanced-rag-using-bedrock-and-sagemaker/embed_algo_costs.csv


,chunking_algo,embedding_seconds,input_tokens,invocation_count,total_token_costs
0,fixed,41.132119,243046,906,0.004861
1,hierarchical,56.310922,0,0,0.000000
2,semantic,131.723036,633504,3958,0.012670


### 1. Create a Knowledge Base
Let's specify  chunking strategy, name and descripotion for Knowledge Base (KB) and create a KB.

In [82]:
model_id = "amazon.titan-embed-text-v2:0"
kb_chunking_strategy = "hierarchical" # ["fixed", "hierarchical", "semantic", "custom"]

In [83]:
kb_name = f"advanced-rag-workshop-{kb_chunking_strategy}-chunking"

kb_description = "Knowledge base using Amazon OpenSearch Service as a vector store"

kb = create_kb(kb_name, kb_description, kb_chunking_strategy, variables, model_id)

{'collectionArn': 'arn:aws:aoss:us-west-2:270597685972:collection/3ethft3xms9as2092ulg', 'vectorIndexName': 'ws-index-hierarchical', 'fieldMapping': {'vectorField': 'vector', 'textField': 'text', 'metadataField': 'text-metadata'}}
{'collectionArn': 'arn:aws:aoss:us-west-2:270597685972:collection/3ethft3xms9as2092ulg', 'vectorIndexName': 'ws-index-hierarchical', 'fieldMapping': {'vectorField': 'vector', 'textField': 'text', 'metadataField': 'text-metadata'}}
{'collectionArn': 'arn:aws:aoss:us-west-2:270597685972:collection/3ethft3xms9as2092ulg', 'vectorIndexName': 'ws-index-hierarchical', 'fieldMapping': {'vectorField': 'vector', 'textField': 'text', 'metadataField': 'text-metadata'}}
Knowledge Base already exists. Retrieving its ID...
Found existing knowledge base with Name: advanced-rag-workshop-hierarchical-chunking and ID: V8EJKFPYTK
OpenSearch Knowledge Response: {
    "createdAt": "2025-05-02 18:04:08.749522+00:00",
    "description": "Knowledge base using Amazon OpenSearch Servic

### 2. Create Datasource for Knowledge Base

In [84]:
data_source_name = f"advanced-rag-example-{kb_chunking_strategy}"

ds_object = create_data_source_for_kb(kb_chunking_strategy, data_source_name, kb, variables)

Found existing data source 'advanced-rag-example-hierarchical'. Deleting it...
Waiting for data source deletion to complete...
Data source deleted successfully.
Creating new data source 'advanced-rag-example-hierarchical' with {'chunkingStrategy': 'HIERARCHICAL', 'hierarchicalChunkingConfiguration': {'levelConfigurations': [{'maxTokens': 1500}, {'maxTokens': 300}], 'overlapTokens': 60}} chunking...
hierarchical chunking data source created successfully.


### 3. Start Ingestion Job for Amazon Bedrock Knowledge base pointing to Amazon OpenSearch

> **Note**: The ingestion process will take approximately 2-3 minutes to complete. During this time, the system is processing your documents by:
> 1. Extracting text from the source files
> 2. Chunking the content according to the defined strategy (Fixed / Semantic / Hierachical / Custom)
> 3. Generating embeddings for each chunk
> 4. Storing the embeddings and associated metadata in a Knowledge Base (KB) in OpenSearch vector database
>
> You'll see status updates as the process progresses. Please wait for the "Ingestion job completed successfully" message before proceeding to the next step.

In [85]:
from time import sleep
ingestion_start_time = datetime.now(UTC)
sleep(3)
create_ingestion_job(kb, ds_object, variables)
sleep(3)
ingestion_end_time = datetime.now(UTC)

Ingestion job started successfully for kb_name = advanced-rag-workshop-hierarchical-chunking and kb_id = V8EJKFPYTK

running...
running...
running...
running...
running...
Job completed successfully



In [86]:
time_taken = (ingestion_end_time-ingestion_start_time).total_seconds()
print(f"time taken to ingest into KB = {fmt_n(time_taken)} seconds")

time taken to ingest into KB = 56.29 seconds


## Embedding LLM Costs
1. Specify model id
2. Specify start and end time
3. Invoke a helper function to query cloud watch
5. Calculate costs (please note that pricing is subject to change per region and over time)

<br>![Embedding LLM Input Token Costs](./Input_token_embedding_llm_costs.png)

In [87]:
vector_store_embedding_cost = get_bedrock_token_based_cost(model_id, ingestion_start_time, ingestion_end_time)
print(json.dumps(vector_store_embedding_cost, indent=4))

{
    "WARNING": "These costs are approximate and directional as of April 2025. They will vary as per region and may change in future. The costs are for Bedrock On-Demand model. If you see zero costs, chances are high that the LLM was not considered in cost calculations when the notebook was prepared. Please use AWS calculator for more accurate cost calculations.",
    "model_id": "amazon.titan-embed-text-v2:0",
    "start_time": "2025-05-02T23:19:04.033033+00:00",
    "end_time": "2025-05-02T23:20:00.320308+00:00",
    "duration in minutes": 0.93812125,
    "input_tokens": 295358,
    "output_tokens": 0,
    "invocation_count": 1157,
    "per million input token costs": 0.02,
    "per million output token costs": 0.0,
    "input token costs": 0.005907160000000001,
    "output token costs": 0.0,
    "total token costs": 0.005907160000000001,
    "average token costs per invocation": 5.105583405358687e-06,
    "token costs per MILLION such invocations": 5.105583405358686
}


In [88]:
# Let's add or update the cost binfo to dataframe. 
# This will help us compare the costs from various chunking strategies visually.
new_row = {
    'chunking_algo': kb_chunking_strategy,
    'embedding_seconds': vector_store_embedding_cost['duration in minutes']*60,
    'input_tokens': vector_store_embedding_cost['input_tokens'],
    'invocation_count': vector_store_embedding_cost['invocation_count'],
    'total_token_costs': vector_store_embedding_cost['total token costs']
}
df_costs = update_or_add_row(df_costs, new_row)
df_costs

Updated existing row for: hierarchical


,chunking_algo,embedding_seconds,input_tokens,invocation_count,total_token_costs
0,fixed,41.132119,243046,906,0.004861
1,hierarchical,56.287275,295358,1157,0.005907
2,semantic,131.723036,633504,3958,0.012670


In [89]:
# Let's save the df
save_df_to_csv(df_costs)

Successfully saved DataFrame to: /home/sagemaker-user/sample-advanced-rag-using-bedrock-and-sagemaker/embed_algo_costs.csv


### 4. Retrieve: Use input query to RETRIEVE chunks from Vector Database
We will use a helper function where you can specify the number of chunks to extract.<br>
The helper function will 1/ generate a vector embedding for the query, 2/ search the vector embedding in the Knowledge Base (KB) vector database, 3/ get the number of chunks specified, 4/ Optionally, you can also specify minimum score for similarity in which case the helper function will get chunks with at least the minimum relevancy.

<b>Warning: After data is ingested into a KB, when you query immediately, the results might be empty because of eventual consistency. If that happens, please wait for a few seconds and then retry.</b>

In [90]:
# Now let's pick the chunks with some minimum relevance score for the same question.
query = "Who is the CEO, CFO, and CTO of Amazon?"

#specify the number of chunks
n_chunks = 5

#Let's specify a minimum similarity score. We should see less chunks retrieved as compared to the previous invocation.
min_score = 0.60

# get chunks from KB
chunks_from_kb = retrieve_from_kb(query, kb, n_chunks, variables, min_score)

print(json.dumps(chunks_from_kb, indent=2))

# You should see less number of chunks retrieved as compared to the previous cell 
# because of the minimum relevance score.

[
  {
    "content": "We also continue to inspect and refine the mechanisms we use to hire, develop, evaluate, and retain our employees. In addition, safety is integral to everything we do at Amazon and we continue to invest in safety improvements such as capital improvements, new safety technology, vehicle safety controls, and engineering ergonomic solutions. Our safety team is dedicated to using the science of safety to solve complex problems and establish new industry best practices. We also provide mentorship and support resources to our employees, and have deployed numerous programs that advance employee engagement, communication, and feedback.     4Table of Contents     Available Information     Our investor relations website is amazon.com/ir and we encourage investors to use it as a way of easily finding information about us. We promptly make available on this website, free of charge, the reports that we file or furnish with the Securities and Exchange Commission (\u201cSEC\u201

In [91]:
# Let's summarize with total chunks, minimum score, maximum score, average score, 
# and lastly the number of chunks with a score more than a specified threshold.
score_threshold = 0.40
score_structure = analyze_chunk_scores_above_threshold(chunks_from_kb, score_threshold)
print(json.dumps(score_structure, indent=4))

{
    "total_chunks": 4,
    "min_score": 0.6470339,
    "max_score": 0.7151378,
    "avg_score": 0.68319585,
    "count_above_threshold": 4
}


### Cost Summary for Running This Notebook
In this notebook, we have used an embedding LLM for two purposes. 
1. Populate a vector store for six PDF files and one CSV file. (7 documents in total)
2. Generate a query embedding.

In [92]:
from IPython.display import display, Markdown
from advanced_rag_utils import embedding_cost_report
from time import sleep
# Marking notebook endtime

sleep(5)
notebook_end_time = datetime.now(UTC)

cost_for_notebook = get_bedrock_token_based_cost(model_id, notebook_start_time, notebook_end_time)

# Your assumptions for your use case:
scenario_number_of_documents = 1000
scenario_number_of_queries = 15000000
 
display(Markdown(embedding_cost_report(vector_store_embedding_cost, cost_for_notebook, scenario_number_of_documents, scenario_number_of_queries)))


#### Scenario
* Number of documents to ingest: 1000
* Number of queries: 15000000

#### Cost Estimation based on the Scenario (USD)
|-| Notebook Cost | Scenario Cost |
|-|-|-|
|VectorStore|0.005907|0.84388|
|Queries|2.8000000000007186e-07|4.2|
|**TOTAL**|0.005907|5.04388|

#### The cost estimation is based on a scenario that the similar documents and queries are multiplied.
        